In [1]:
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

from typing import Tuple
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import pandas as pd

%matplotlib inline
import logging

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
from cycler import cycler
from PIL import Image
from torchvision import datasets, transforms
import tqdm

import pytorch_metric_learning
import pytorch_metric_learning.utils.logging_presets as logging_presets
from pytorch_metric_learning import losses, miners, samplers, testers, trainers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s" % pytorch_metric_learning.__version__)

INFO:root:VERSION 2.3.0


In [2]:
margin_param=0.2

In [3]:
x = pd.read_csv('../dataset/X.csv')
y = pd.read_csv('../dataset/y.csv')

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class MyNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        return x

input_size = x.shape[1]
hidden_size1 = 64
hidden_size2 = 32
output_size = 20


model = MyNN(input_size, hidden_size1, hidden_size2, output_size)

In [5]:
class ItemDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x.values,dtype=torch.float32,requires_grad=True)
        self.y = torch.tensor(y.values).flatten()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = ItemDataset(x, y)

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
loss = losses.TripletMarginLoss(margin=margin_param)
miner = miners.TripletMarginMiner(margin=margin_param, type_of_triplets="hard")
sampler = samplers.MPerClassSampler(y, m=4, length_before_new_iter=len(train_dataset))

batch_size = 128
num_epochs = 10
dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)

losses = []
for epoch in tqdm.tqdm(range(num_epochs)):   
    local_loss = 0
    for i, (data,label) in enumerate(dataloader):
        optimizer.zero_grad()
        embeddings = model(data)
        hard_pairs = miner(embeddings, label)
        loss_value = loss(embeddings, label, hard_pairs)
        loss_value.backward()
        local_loss += loss_value.item()
        optimizer.step()
        
    average_loss = local_loss / len(dataloader)
    losses.append(average_loss)

 40%|████      | 4/10 [25:41<37:51, 378.57s/it]  

In [ ]:
torch.save(model.state_dict(), '../dataset/model.pth')

In [ ]:
plt.plot(losses)
plt.show()